In [37]:
import numpy as np
X = np.load("X.npy")
y = np.load("y.npy")



In [38]:
print(np.isnan(X).any(), np.isnan(y).any())
print(np.isinf(X).any(), np.isinf(y).any())


True False
False False


In [39]:
X = np.nan_to_num(X)
y = np.nan_to_num(y)

In [40]:
print("X min/max:", X.min(), X.max())
print("y min/max:", y.min(), y.max())


X min/max: -17531.0 312972.0
y min/max: 0.0 127.301369863


In [41]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_reshaped = X.reshape(-1, X.shape[-1])  # объединяем временные шаги
X_scaled = scaler.fit_transform(X_reshaped)
X = X_scaled.reshape(X.shape)  # возвращаем форму


In [42]:
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

dataset = CustomDataset(X, y)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)


In [ ]:
import torch.nn as nn

#  Модель (many-to-one) 
class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super().__init__()
        self.rnn = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out, (h_n, c_n) = self.rnn(x)
        # берём последнее скрытое состояние (many-to-one)
        last_hidden = h_n[-1]
        out = self.fc(last_hidden)
        return out


In [ ]:
# Параметры модели 
input_dim = X.shape[2]
hidden_dim = 64
num_layers = 1
output_dim = 1  # регрессия (если классификация — заменить на число классов)

model = RNNModel(input_dim, hidden_dim, num_layers, output_dim)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)



In [45]:
from tqdm import tqdm
# === 5. Обучение ===
num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for X_batch, y_batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        optimizer.zero_grad()
        y_pred = model(X_batch).squeeze()
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f"Epoch [{epoch+1}/{num_epochs}] Loss: {epoch_loss/len(train_loader):.4f}")

Epoch 1/20: 100%|██████████| 158/158 [00:00<00:00, 182.28it/s]


Epoch [1/20] Loss: 1572.2959


Epoch 2/20: 100%|██████████| 158/158 [00:00<00:00, 221.49it/s]


Epoch [2/20] Loss: 862.6185


Epoch 3/20: 100%|██████████| 158/158 [00:00<00:00, 228.32it/s]


...

Epoch [18/20] Loss: 181.7990


Epoch 19/20: 100%|██████████| 158/158 [00:00<00:00, 224.32it/s]


Epoch [19/20] Loss: 183.7201


Epoch 20/20: 100%|██████████| 158/158 [00:00<00:00, 216.73it/s]

Epoch [20/20] Loss: 178.7394


In [46]:
# === 6. Пример предсказания ===
model.eval()
with torch.no_grad():
    y_pred = model(torch.tensor(X[:5], dtype=torch.float32)).squeeze()
    print("Predictions:", y_pred)
    print("True:", y[:5])

Predictions: tensor([38.6009, 39.4441, 43.0973, 32.9326, 44.8101])
True: [42.79726027 29.23561644 41.67945205 24.91232877 43.35068493]
